#### Converting to CSV

In [ ]:
import os
import glob
import pandas as pd
from netCDF4 import Dataset

# Define the paths to the 'Data' and 'CSV' folders
data_folder = "../Level1-data/"
csv_folder = "../Level1-mg-CSV-11days/"

# Create the 'CSV' folder if it doesn't exist
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

# Check for files with .nc extension in the 'Data' folder
nc_files = glob.glob(os.path.join(data_folder, "*.nc"))

if not nc_files:
    print("No .nc files found in the 'Data' folder.")
else:
    # Loop through the .nc files and convert them to CSV with batch processing
    chunk_size = 5000
    for nc_file in nc_files:
        file_name = os.path.basename(nc_file)  # Get the file name without path
        csv_file = os.path.join(csv_folder, os.path.splitext(file_name)[0] + ".csv")

        try:
            # Open the .nc file using netCDF4
            nc = Dataset(nc_file, 'r')

            # Create a Pandas DataFrame in chunks and save to CSV
            for i in range(0, len(nc.dimensions['time']), chunk_size):
                chunk_end = min(i + chunk_size, len(nc.dimensions['time']))
                variable_data = {}

                # Extract variables and their data for the current chunk
                for var_name, var in nc.variables.items():
                    variable_data[var_name] = var[i:chunk_end]

                # Create a Pandas DataFrame from the variable data
                df = pd.DataFrame(variable_data)

                # Save the DataFrame as a CSV file
                if i == 0:
                    df.to_csv(csv_file, mode='w', index=False)
                else:
                    df.to_csv(csv_file, mode='a', header=False, index=False)

            print(f"Converted {file_name} to {os.path.basename(csv_file)}")

        except Exception as e:
            print(f"Error converting {file_name}: {str(e)}")

# Now, create a Pandas DataFrame from the CSV files in the 'CSV' folder
csv_files = glob.glob(os.path.join(csv_folder, "*.csv"))

if not csv_files:
    print("No CSV files found in the 'CSV' folder.")
else:
    # Concatenate the CSV files into a Pandas DataFrame
    df = pd.concat(map(pd.read_csv, csv_files))
    print("Concatenation of CSV files completed.")

# Now, 'df' contains the combined data from all CSV files.


In [ ]:
import pandas as pd

common_times = df1[df1['time'].isin(df2['time'])]

merged_df = common_times.merge(df1, on='time')

merged_df.to_csv('../Dataset/final_mg.csv', index=False)

print("Success")